<a href="https://colab.research.google.com/github/shaoanlu/autompc/blob/main/examples/0_MainDemo_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AutoMPC Demo

Welcome!  This notebook demonstrates the core features of AutoMPC.

AutoMPC is designed to simplify the process of creating a controller for a robot system with unknown dynamics.
A standard approach to solving this problem is 1) use a System ID algorithm to produce a model of the system dynamics
from data, 2) design an objective which captures the task you want to solve, 3) use an optimization algorithm to solve for a control sequence with respect  to the model and objective function.  AutoMPC provides a toolbox of algorithms for  all three steps, and automates the process of hyperparameter selection for each component.

In this notebook, we will use the cart-pole swing-up task as an example.  Although we know the ground truth dynamics for the cart-pole system, we will demonstrate how to work with an unknown dynamical system by using a multi-layer perceptron (MLP) to learn the dynamics from data.  We will design our model predictive controller using a standard quadratic objective function and an iterative LQR optimizer.

In [2]:
!ls

autompc				    sample_data
autompc-output_2022-08-12T17:25:08  smac3-output_2022-08-12_17:25:08_836437
autompc-output_2022-08-12T17:29:51  smac3-output_2022-08-12_17:29:52_238106
autompc-output_2022-08-12T17:30:19  smac3-output_2022-08-12_17:30:20_109711
autompc-output_2022-08-12T17:36:34  smac3-output_2022-08-12_17:36:35_246017
autompc-output_2022-08-12T17:37:07  smac3-output_2022-08-12_17:37:07_377188
autompc-output_2022-08-12T17:37:17  smac3-output_2022-08-12_17:37:17_362011
autompc-output_2022-08-12T17:37:24  smac3-output_2022-08-12_17:37:24_257633
autompc-output_2022-08-12T17:38:21  smac3-output_2022-08-12_17:38:21_774737
autompc-output_2022-08-12T17:40:57  smac3-output_2022-08-12_17:40:57_544230
autompc-output_2022-08-12T17:45:45  smac3-output_2022-08-12_17:45:45_344794
autompc-output_2022-08-12T17:46:30  smac3-output_2022-08-12_17:46:30_824039
autompc-output_2022-08-12T17:47:10  smac3-output_2022-08-12_17:47:10_439108
Miniconda3-4.5.4-Linux-x86_64.sh


In [23]:
%env PYTHONPATH=

env: PYTHONPATH=


In [24]:
%%bash
MINICONDA_INSTALLER_SCRIPT=Miniconda3-4.5.4-Linux-x86_64.sh
MINICONDA_PREFIX=/usr/local
wget https://repo.continuum.io/miniconda/$MINICONDA_INSTALLER_SCRIPT
chmod +x $MINICONDA_INSTALLER_SCRIPT
./$MINICONDA_INSTALLER_SCRIPT -b -f -p $MINICONDA_PREFIX

PREFIX=/usr/local
installing: python-3.6.5-hc3d631a_2 ...
installing: ca-certificates-2018.03.07-0 ...
installing: conda-env-2.6.0-h36134e3_1 ...
installing: libgcc-ng-7.2.0-hdf63c60_3 ...
installing: libstdcxx-ng-7.2.0-hdf63c60_3 ...
installing: libffi-3.2.1-hd88cf55_4 ...
installing: ncurses-6.1-hf484d3e_0 ...
installing: openssl-1.0.2o-h20670df_0 ...
installing: tk-8.6.7-hc745277_3 ...
installing: xz-5.2.4-h14c3975_4 ...
installing: yaml-0.1.7-had09818_2 ...
installing: zlib-1.2.11-ha838bed_2 ...
installing: libedit-3.1.20170329-h6b74fdf_2 ...
installing: readline-7.0-ha6073c6_4 ...
installing: sqlite-3.23.1-he433501_0 ...
installing: asn1crypto-0.24.0-py36_0 ...
installing: certifi-2018.4.16-py36_0 ...
installing: chardet-3.0.4-py36h0f667ec_1 ...
installing: idna-2.6-py36h82fb2a8_1 ...
installing: pycosat-0.6.3-py36h0a5515d_0 ...
installing: pycparser-2.18-py36hf9f622e_1 ...
installing: pysocks-1.6.8-py36_0 ...
installing: ruamel_yaml-0.15.37-py36h14c3975_2 ...
installing: six-1.11

--2022-08-12 18:32:49--  https://repo.continuum.io/miniconda/Miniconda3-4.5.4-Linux-x86_64.sh
Resolving repo.continuum.io (repo.continuum.io)... 104.18.200.79, 104.18.201.79, 2606:4700::6812:c94f, ...
Connecting to repo.continuum.io (repo.continuum.io)|104.18.200.79|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://repo.anaconda.com/miniconda/Miniconda3-4.5.4-Linux-x86_64.sh [following]
--2022-08-12 18:32:49--  https://repo.anaconda.com/miniconda/Miniconda3-4.5.4-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.131.3, 104.16.130.3, 2606:4700::6810:8303, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.131.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 58468498 (56M) [application/x-sh]
Saving to: ‘Miniconda3-4.5.4-Linux-x86_64.sh’

     0K .......... .......... .......... .......... ..........  0% 44.9M 1s
    50K .......... .......... .......... .......... ..........  0%

In [25]:
%%bash
conda install --channel defaults conda python=3.6 --yes
conda update --channel defaults --all --yes

Solving environment: ...working... done

## Package Plan ##

  environment location: /usr/local

  added / updated specs: 
    - conda
    - python=3.6


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    conda-4.10.3               |   py36h06a4308_0         3.1 MB
    urllib3-1.26.8             |     pyhd3eb1b0_0         100 KB
    openssl-1.1.1q             |       h7f8727e_0         3.8 MB
    wheel-0.37.1               |     pyhd3eb1b0_0          31 KB
    pycparser-2.21             |     pyhd3eb1b0_0          94 KB
    colorama-0.4.4             |     pyhd3eb1b0_0          21 KB
    ruamel_yaml-0.15.100       |   py36h27cfd23_0         268 KB
    cffi-1.14.6                |   py36h400218f_0         224 KB
    python-3.6.13              |       h12debd9_1        32.5 MB
    brotlipy-0.7.0             |py36h27cfd23_1003         349 KB
    libffi-3.3                 |       he6710b0_2   

conda-4.10.3         |  3.1 MB | ########## | 100% 
urllib3-1.26.8       |  100 KB | ########## | 100% 
openssl-1.1.1q       |  3.8 MB | ########## | 100% 
wheel-0.37.1         |   31 KB | ########## | 100% 
pycparser-2.21       |   94 KB | ########## | 100% 
colorama-0.4.4       |   21 KB | ########## | 100% 
ruamel_yaml-0.15.100 |  268 KB | ########## | 100% 
cffi-1.14.6          |  224 KB | ########## | 100% 
python-3.6.13        | 32.5 MB | ########## | 100% 
brotlipy-0.7.0       |  349 KB | ########## | 100% 
libffi-3.3           |   54 KB | ########## | 100% 
conda-package-handli |  946 KB | ########## | 100% 
ld_impl_linux-64-2.3 |  732 KB | ########## | 100% 
zlib-1.2.12          |  130 KB | ########## | 100% 
charset-normalizer-2 |   33 KB | ########## | 100% 
certifi-2021.5.30    |  141 KB | ########## | 100% 
xz-5.2.5             |  389 KB | ########## | 100% 
tk-8.6.12            |  3.3 MB | ########## | 100% 
pysocks-1.7.1        |   30 KB | ########## | 100% 
pip-21.2.2  

In [26]:
import sys
_ = (sys.path.append("/usr/local/lib/python3.6/site-packages"))

In [27]:
!git clone https://github.com/mechmotum/cyipopt.git

Cloning into 'cyipopt'...
remote: Enumerating objects: 2265, done.
remote: Counting objects: 100% (300/300), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 2265 (delta 275), reused 258 (delta 258), pack-reused 1965
Receiving objects: 100% (2265/2265), 1.52 MiB | 13.17 MiB/s, done.
Resolving deltas: 100% (1391/1391), done.


In [28]:
%cd cyipopt

/content/autompc/cyipopt


In [29]:
!conda install -c conda-forge cyipopt
!sudo apt install build-essential pkg-config python-dev cython python-numpy coinor-libipopt1v5 coinor-libipopt-dev
!python setup.py build
!python setup.py install

Solving environment: | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | done


==> WARNING: A newer version of conda exists. <==
  current version: 4.10.3
  latest version: 4.13.0

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - cyipopt


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ampl-mp-3.1.0              |    h2cc385e_1005         1.2 MB  conda-forge
    ca-certificates-2022.6.15  |       ha878542_0         149 KB  conda-forge
    cyipopt-1.1.0              |   py36h68cfc00_0         144 KB  conda-forge
    future-0.18.2              |   py36h5fab9bb_3         716 KB  conda-forge
    ipopt-3.14.3               |       h7ede334_0         1.2 MB  conda-forge
    libblas-3.9.0              |15_linux64_

In [6]:
!conda install -c conda-forge ipopt cyipopt

Solving environment: \ | / - \ | / - \ | / - \ | / - done


==> WARNING: A newer version of conda exists. <==
  current version: 4.10.3
  latest version: 4.13.0

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.



In [5]:
%cd ..

/content


In [6]:
!ls

sample_data  SMAC3


In [7]:
!git clone https://github.com/williamedwards/autompc.git

Cloning into 'autompc'...
remote: Enumerating objects: 4129, done.
remote: Counting objects: 100% (615/615), done.
remote: Compressing objects: 100% (234/234), done.
remote: Total 4129 (delta 390), reused 592 (delta 380), pack-reused 3514
Receiving objects: 100% (4129/4129), 7.74 MiB | 21.95 MiB/s, done.
Resolving deltas: 100% (2875/2875), done.


In [8]:
%cd autompc

/content/autompc


In [9]:
!ls

assets	 docs	   LICENSE    requirements.txt	tests
autompc  examples  README.md  setup.py


In [10]:
!git checkout origin/0.2-dev

Note: checking out 'origin/0.2-dev'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by performing another checkout.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -b with the checkout command again. Example:

  git checkout -b <new-branch-name>

HEAD is now at 585dab2 Fixed convention used by ThresholdCost so that it reports goals properly


In [11]:
#!pip install -r requirements.txt
!pip install -e .

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/autompc
  Running setup.py develop for autompc


In [ ]:
%cd ..

In [1]:
!git clone https://github.com/automl/SMAC3.git

Cloning into 'SMAC3'...
remote: Enumerating objects: 56617, done.
remote: Counting objects: 100% (1258/1258), done.
remote: Compressing objects: 100% (393/393), done.
remote: Total 56617 (delta 905), reused 1130 (delta 838), pack-reused 55359
Receiving objects: 100% (56617/56617), 61.39 MiB | 20.03 MiB/s, done.
Resolving deltas: 100% (45377/45377), done.


In [2]:
%cd SMAC3

/content/SMAC3


In [3]:
!pip install .

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing /content/SMAC3
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 4.7 MB 5.3 MB/s 
     |████████████████████████████████| 46 kB 2.7 MB/s 
     |████████████████████████████████| 4.4 MB 41.0 MB/s 
  Created wheel for smac: filename=smac-1.4.0-py3-none-any.whl size=395493 sha256=58d

In [4]:
from smac.facade.smac_mf_facade import SMAC4MF

In [13]:
!pip install gpytorch pysindy ConfigSpace

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 361 kB 5.0 MB/s 
     |████████████████████████████████| 104 kB 57.5 MB/s 


## Set-Up

First, we will import the autompc library and other necessary dependancies.

In [1]:
import autompc as ampc
import numpy as np
import matplotlib.pyplot as plt
%matplotlib notebook

Loading AutoMPC...
Finished loading AutoMPC


The benchmarks module provides some tools for quickly building example problems.  For a list of available benchmarks, see [here](https://autompc.readthedocs.io/en/latest/source/benchmarks.html#available-benchmarks).  Here we will import the cart-pole benchmark

In [2]:
from autompc.benchmarks import CartpoleSwingupV2Benchmark

benchmark = CartpoleSwingupV2Benchmark()

The `system` object defines the observation and contorl dimensions of the cartpole, while the `task` defines the task we want to solve.  Here, we will just get these from the benchmark, but for more details on systems see example [1. Basics](https://github.com/williamedwards/autompc/tree/main/examples) and for more details on tasks see example [3. Controllers and Tasks](https://github.com/williamedwards/autompc/tree/main/examples).

In [3]:
# Get system and task specification
system = benchmark.system
task   = benchmark.task

We also need a dataset of trajectories sampled from the system to use for system identification and tuning.  On a real system, we would collect these from the robot, but here we'll use another method from our benchmark to generate the dataset.  The default option is to generate trajectories using uniform random controls.

In [4]:
# Generate benchmark dataset
trajs = benchmark.gen_trajs(seed=100, n_trajs=100, traj_len=200)

The benchmark also provides capabilities to visualize trajectories.  We can use this to visualize one of the trajectories in our training set.  Since the dataset was generated using uniform random controls, we see that this trajectory clearly does not accomplish the cart-pole swing-up task.  (Note: the animation may take a minute to generate).

In [5]:
from IPython.display import HTML

# fig = plt.figure()
# ax = fig.gca()
# anim = benchmark.visualize(fig, ax, trajs[1])
# HTML(anim.to_html5_video())

<IPython.core.display.Javascript object>

Next, we need to create an MPC to be tuned.  Here, we will use the most general form, AutoSelectController, which allows AutoMPC to automatically select between all of its system ID, optimization, and OCP generation algorithms.

In [6]:
from autompc import AutoSelectController
controller = AutoSelectController(system)

We can view the tunable hyperparameters of the controller.  Since AutoSelectController chooses between all possible models and optimizers, the hyperparameter space is quite large!

In [7]:
controller.set_ocp(benchmark.task.get_ocp())
print(controller.get_config_space())

Forbidding model MLP to be used with LQR due to property is_linear
Forbidding model SINDy to be used with LQR due to property is_linear
Checking compatibility with OCP
Requiring cost transformer for ocp to be used with IterativeLQR due to property is_twice_diff
Requiring cost transformer for ocp to be used with LQR due to property is_quad
Configuration space object:
  Hyperparameters:
    ARX:history, Type: UniformInteger, Range: [1, 10], Default: 4
    DirectTranscription:horizon, Type: UniformInteger, Range: [1, 30], Default: 10
    GaussRegTransformer:reg_weight, Type: UniformFloat, Range: [0.001, 10000.0], Default: 1.0, on log-scale
    IterativeLQR:frequency, Type: UniformInteger, Range: [1, 5], Default: 1
    IterativeLQR:horizon, Type: UniformInteger, Range: [5, 25], Default: 20
    IterativeLQR:max_iter, Type: UniformInteger, Range: [10, 50], Default: 20
    Koopman:lasso_alpha, Type: UniformFloat, Range: [1e-10, 100.0], Default: 1.0, on log-scale
    Koopman:method, Type: Cate

## Tuning

Once we have initialized the pipeline and its factories, we can set up a tuner to automatically search over the pipeline's configuration space.  Since we assume we don't have access to the ground truth dynamics, we train a surrogate dynamics model, which is used as a simulator to evaluate configurations.

Here we pass an `MLP` instance to the tuner to be used to train the surrogate dynamics model.  The `surrogate_split` controls what proportion of the data will be used for training the surrogate dynamics model vs training the system ID model.

This is an example of full pipeline which searches the configuration space of all pipeline components simultaneously.  (NOTE: This takes quite a while to run.  Skip this cell and run the next one instead to load a cached result).

In [9]:
from autompc.tuning import ControlTuner
from autompc.sysid import MLP, SINDy, Koopman, ApproximateGPModel, ARX

tuner = ControlTuner(surrogate=MLP(system))

We can now run the tuning porcess.  In addition to the pipeline, we pass the task and the trajectory dataset.  The task specification is used to evaluate the trajectories simulated with the surrogate dynamics model.  AutoMPC uses the `smac3` package for Bayesian optimization to search the configuration space.

Here, we pass the ground truth dynamics to the tuner so that we can measure the true performance over time.  However, we would not have this for a real robot, so it's not used for selecting the configuration.

**Note:** The tuning process can take 5 or more hours to run depending on the hardware available.  To load a cached result instead of running this yourself, skip this cell and run the next one instead.

In [ ]:
tuned_controller, tune_result = tuner.run(controller, task, trajs, n_iters=100, rng=np.random.default_rng(100),
                                   truedyn=benchmark.dynamics)
                                   #restore_dir="autompc-output_2022-02-10T17:10:43")

/usr/local/lib/python3.7/dist-packages/smac/intensification/parallel_scheduling.py:155: UserWarning: Hyperband is executed with 1 workers only. Consider to use pynisher to use all available workers.
  f"{self.__class__.__name__} is executed with {num_workers} workers only. "


------------------------------------------------------------------
Beginning surrogate tuning with model class MLP
------------------------------------------------------------------
 40%|███▉      | 79/200 [00:15<00:23,  5.17it/s]

Run the below cell to load a cached tune result

In [40]:
#import os, pickle
#current_folder = globals()['_dh'][0]
#fn = os.path.join(current_folder, "../assets/cached_tunes/cartpole_tune_result.pkl")
#with open(fn, "rb") as f:
#    tune_result = pickle.load(f)
#inc_cfg = tune_result.inc_cfg
#controller, cost, model = pipeline(inc_cfg, task, trajs)

ModuleNotFoundError: ignored

We can then plot the performance of the best controller found so far over the iterations of the tuning process.  We plot both the performance as evaluated with the surrogate dynamics and the true dynamics performance.  Lower scores are better, so after 100 iterations, both the surrogate and true performance converge to a good solution.

In [ ]:
from autompc.graphs import TuningCurveGraph
import matplotlib.pyplot as plt

graph = TuningCurveGraph()

fig = plt.figure()      
ax = fig.gca()
graph(ax, tune_result)
ax.set_title("Cart-Pole Tuning Curve")
plt.show()

We can also view the configurations selected by the tuner.

In [ ]:
tune_result.inc_cfg

Now that our tuning process has given us a controller, we can run it to simulate a trajectory.  For more information on how to work with controllers, see example [3. Controllesr .

In [ ]:
traj = ampc.simulate(tuned_controller, init_obs=benchmark.task.get_init_obs(), max_steps=200, dynamics=benchmark.dynamics)

In [ ]:
fig = plt.figure()
ax = fig.gca()
anim = benchmark.visualize(fig, ax, traj)
HTML(anim.to_html5_video())

In [ ]:
import pickle
with open("../assets/cached_tunes/cartpole_v2_result_01.pkl", "wb") as f:
    pickle.dump(tune_result, f)